# Activebrain_sandbox


In [1]:
import pandas as pd
import numpy as np
import os
import datajoint as dj
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
matplotlib.style.use('ggplot')
%matplotlib inline

<a id='connect'></a>
# Connect to the datajoint00 database
Username and password are same as Princeton login.

**Note**: You need to be on campus or using a VPN to connect to the database.

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [3]:
schema = dj.Schema('ahoag_test_django')

In [30]:
# db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_spockadmin = dj.create_virtual_module('ahoag_spockadmin_demo','ahoag_spockadmin_demo')
db_lightsheet = dj.create_virtual_module('a','u19lightserv_lightsheet')

## Sandbox begins

In [10]:
db_lightsheet.Request() & {'username':'willmore'}

username user in the lab,request_name,requested_by user in the lab,auditor user in the lab,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,testing,is_archival,raw_data_retention_preference,sent_processing_email
willmore,201106_willmore,willmore,None,2020-11-05,12:42:00,witten/falkner,willmore@princeton.edu,find fiber tracts. register brains to allen. compare to previous batches of registered brains.,mouse,8,0,0,important,0
willmore,20190304_LW_m340,willmore,None,2019-03-02,11:27:16,Falkner/Witten,willmore@princeton.edu,Detect placement of optical fiber tip,mouse,7,0,1,None,0
willmore,20190510_lw_059,willmore,None,2019-05-01,17:25:35,Falkner/Witten,willmore@princeton.edu,fiber tip placement,mouse,11,0,1,None,0
willmore,20190809_lw_32,willmore,None,2019-08-09,15:05:24,Falkner/Witten,willmore@princeton.edu,Fiber tip placement,mouse,8,0,1,None,0
willmore,"20191001_LW_10,_11,12,_13,_54,_55,_60,_61,_70,_71,_392",willmore,None,2019-09-30,12:59:57,Witten,willmore@princeton.edu,registration and location of probe insertion,mouse,11,0,1,None,0


In [ ]:
db_lightsheet.Request.ClearingBatch() & {'username':'hboele'}

In [ ]:
db_lightsheet.Request.ClearingBatchSample() & {'username':'hboele'}

In [ ]:
db_lightsheet.Request.IdiscoPlusImmunoClearing()

In [ ]:
db_lightsheet.Request.ImagingBatchSample() & {'username':'hboele'}

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
unique_contents = dj.U('jobid_step0','username',).aggr(
    job_contents,timestamp='max(timestamp)')*job_contents

In [ ]:
unique_contents

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = request_contents.ClearingBatch()
combined_contents = request_contents *clearing_batch_contents

In [ ]:
combined_contents

In [ ]:
username='soline'

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & {'username':username}

In [ ]:
db_lightsheet.Request.ImagingChannel() & {'username':username}

In [ ]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

In [ ]:
restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj('imager',
                              'imaging_request_date_submitted',
                              'imaging_request_time_submitted',
                              'imaging_performed_date',
                              'imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
                            'processing_progress',
                            'processor')

In [ ]:
replicated_args = dict(is_archival='is_archival',number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted')

In [ ]:
sample_joined_contents = dj.U('username','request_name').aggr(
        request_contents * clearing_batch_contents,
        is_archival='is_archival',
        number_of_samples='number_of_samples',
        number_in_batch='number_in_batch',
        description='description',
        species='species',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)',
        n_cleared='CONVERT(SUM(IF(clearing_progress="complete",number_in_batch,0)),char)').proj(
        **replicated_args,
            fraction_cleared='CONCAT(n_cleared,"/",CONVERT(number_of_samples,char))')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
    imaging_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
    total_imaging_requests='CONVERT(COUNT(*),char)',
    keep_all_rows=True
    ).proj(**replicated_args,
        fraction_cleared='fraction_cleared',
        # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
        fraction_imaged='IF(n_imaged is NULL,"0/0",CONCAT(n_imaged,"/",total_imaging_requests))' 
        )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
    processing_request_contents,
    **replicated_args,
    fraction_cleared='fraction_cleared',
    fraction_imaged='fraction_imaged',
    n_processed='CONVERT(SUM(processing_progress="complete"),char)',
    total_processing_requests='CONVERT(COUNT(processing_progress),char)',
    keep_all_rows=True
    ).proj(
        **replicated_args,
        fraction_cleared='fraction_cleared',
        fraction_imaged='fraction_imaged',
        fraction_processed='IF(n_processed is NULL,"0/0",CONCAT(n_processed,"/",total_processing_requests))' 
        )

In [ ]:
processing_joined_contents

## Debugging all samples with auditor

In [ ]:
request_contents = db_lightsheet.Request()
request_contents = request_contents.proj('description','species','number_of_samples',
        'is_archival','auditor',
        datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
processing_request_contents = db_lightsheet.Request.ProcessingRequest()

restrict_OR_list = [{"username":username},{"auditor":username}]
request_contents = request_contents & restrict_OR_list
sample_contents = (request_contents*sample_contents & restrict_OR_list).proj()

clearing_batch_contents = (request_contents*clearing_batch_contents & restrict_OR_list).proj(
    'clearer','clearing_progress','link_to_clearing_spreadsheet')
imaging_request_contents = (request_contents*imaging_request_contents & restrict_OR_list).proj(
    'imager','imaging_progress')
processing_request_contents = (request_contents*processing_request_contents & restrict_OR_list).proj(
    'processor','processing_progress')
sample_joined_contents = request_contents * sample_contents * clearing_batch_contents
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',datetime_submitted='datetime_submitted',
        clearer='clearer',
        clearing_progress='clearing_progress',
        imager='imager',imaging_progress='imaging_progress',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )

In [ ]:
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
processing_joined_contents

In [ ]:
contents = db_lightsheet.Request() & [{"username":username},{"auditor":username}]
contents

In [ ]:
imaging_batch_restrict_dict = dict(username=username,request_name=request_name,
    imaging_batch_number=imaging_batch_number,
    imaging_request_number=imaging_request_number)
sample_contents = db_lightsheet.Request.ImagingBatchSample() & imaging_batch_restrict_dict

In [ ]:
sample_contents

In [ ]:
restrict_dict = {'username':'lightserv-test','request_name':'test_new_imaging_request'}
db_lightsheet.Request() & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingRequest & restrict_dict

In [ ]:
db_lightsheet.Request.ImagingBatch() & restrict_dict

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)')

In [ ]:
imaging_request_contents = dj.U('username','request_name',
                                'imaging_batch_number',
                                'imaging_request_number').aggr(
    imaging_request_contents,clearer='clearer',
    clearing_progress='clearing_progress',
    datetime_submitted='datetime_submitted',
    imaging_progress='imaging_progress',imager='imager',
    species='species',number_in_imaging_batch='number_in_imaging_batch',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
imaging_request_contents & restrict_dict

In [ ]:
contents_being_imaged = imaging_request_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

In [ ]:
contents_being_imaged & restrict_dict

In [ ]:
imaging_request_contents & 'imaging_request_number=2'

In [ ]:
sample_restrict_dict = \
    {'sample_name': 'test-001', 'imaging_request_number': '2', 'imaging_batch_number': '1'}

In [ ]:
db_lightsheet.Request.ClearingBatchSample()

In [ ]:
(db_lightsheet.Request() & 'username="pbibawi"').fetch('request_name')

In [ ]:
username='willmore'
request_name='201106_willmore'
sample_name = '201106_willmore-303'
imaging_request_number=1
processing_request_number=1
processing_restrict_dict = dict(username=username,request_name=request_name,
                sample_name=sample_name,imaging_request_number=imaging_request_number,
                processing_request_number=processing_request_number)

processing_request_contents = db_lightsheet.Request.ProcessingRequest() & processing_restrict_dict

processing_resolution_request_contents = db_lightsheet.Request.ProcessingResolutionRequest() & \
    processing_restrict_dict

processing_channel_contents = db_lightsheet.Request.ProcessingChannel() & processing_restrict_dict
image_resolution='1.1x'
processing_channel_contents_this_resolution = (processing_channel_contents & \
            f'image_resolution="{image_resolution}"')
restrict_dict = {'channel_name':'488','ventral_up':0}
this_processing_channel_content = (processing_channel_contents_this_resolution & \
                    restrict_dict)
this_processing_channel_content

In [ ]:
username='willmore'
request_name='201106_willmore'
imaging_batch_number=1
imaging_request_number=1

restrict_dict = dict(username=username,request_name=request_name,
                imaging_batch_number=imaging_batch_number)

channel_contents_all_samples = (db_lightsheet.Request.ImagingChannel() & f'request_name="{request_name}"' & \
    f'username="{username}"' )
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    restrict_dict
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() & \
    restrict_dict
imaging_resolution_contents

In [ ]:
channel_contents_all_samples

In [ ]:
image_resolutions_no_processing = ["2x","3.6x","15x"]

In [ ]:
batch_unique_image_resolutions = set(imaging_resolution_contents.fetch('image_resolution'))
# batch_unique_image_re?solutions = set(['1.1x','3.6x'])

In [ ]:
have_processing_requests = any([x not in image_resolutions_no_processing for x in batch_unique_image_resolutions])
have_processing_requests

In [ ]:
set(imaging_request_contents.fetch('sample_name'))


In [ ]:
sample_names = set(imaging_resolution_contents.fetch('sample_name'))
sample_names

In [ ]:
data_rootpath = '/jukebox/LightSheetData/lightserv'

In [ ]:
message_body = ('Hello!\n\nThis is an automated email sent from lightserv, '
                        'the Light Sheet Microscopy portal at the Histology and Brain Registration Core Facility. '
                        'The raw data your request:\n'
                        f'request_name: "my-request"\n'
                        f'are now available on bucket here: \n\n')
for sample_name in sample_names:
    sample_basepath = os.path.join(data_rootpath,username,request_name,
    sample_name,f'imaging_request_number_{imaging_request_number}',  
    'rawdata')
    channel_contents_this_sample = channel_contents_all_samples & f'sample_name="{sample_name}"' 
    message_body += f'Sample name: {sample_name}:\n'
    for channel_dict in channel_contents_this_sample:
        channel_name = channel_dict['channel_name']
        image_resolution = channel_dict['image_resolution']
        ventral_up = channel_dict['ventral_up']
        rawdata_subfolder = channel_dict['rawdata_subfolder']
        if ventral_up:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}_ventral_up',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name} (ventral up): {rawdata_fullpath}\n'
        else:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name}: {rawdata_fullpath}\n'
    message_body += '\n'
message_body += '\n'
if have_processing_requests:
    message_body += ('To start processing your data, '
            f'go to the processing management GUI: /some/path '
            'and find your sample to process.\n\n')
message_body += 'Thanks,\n\nThe Core Facility'
print(message_body)

In [ ]:
this_processing_channel_content_dict = this_processing_channel_content.fetch1()
this_processing_channel_content_dict

In [ ]:
processing_channel_contents_this_resolution

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest().fetch('atlas_name',limit=1)[0]

In [ ]:
processing_resolution_request_contents

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
imaging_channel_contents = db_lightsheet.Request.ImagingChannel()

In [ ]:
imaging_channel_contents & 'ventral_up=1'

In [ ]:
processing_resolution_contents = db_lightsheet.Request.ProcessingResolutionRequest() & {'username':'willmore'}
processing_resolution_contents 

In [ ]:
dj.Table._update(processing_resolution_contents,'atlas_name','allen_2017')

In [ ]:
clearing_batch_contents

In [ ]:
combined_contents = (clearing_batch_contents * request_contents).proj(
'description','number_in_batch','expected_handoff_date',
'clearing_protocol','species',
'clearer','clearing_progress','clearing_protocol','antibody1','antibody2',
datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
''' First get all entities that are currently being cleared '''

In [ ]:
idiscoplus_cleared_contents = combined_contents & 'clearing_protocol="iDISCO+_immuno"' & 'clearing_progress="complete"'
idiscoplus_cleared_contents

In [ ]:
df_idiscoplus_cleared = (idiscoplus_cleared_contents).fetch(
format='frame',order_by='datetime_submitted DESC')

In [ ]:
df_idiscoplus_cleared.to_csv("idiscoplus_already_cleared_asof_2020Nov12.csv")

In [ ]:
idisco_contents = db_lightsheet.Request.IdiscoPlusClearing()
idisco_contents

In [ ]:
idisco_website_cleared_contents = idisco_contents * idiscoplus_cleared_contents
idisco_website_cleared_contents

In [ ]:
idisco_results=idisco_website_cleared_contents.fetch(as_dict=True)
idisco_results

In [ ]:
df_idisco=pd.DataFrame(idisco_results)
df_idisco

In [ ]:
df_idisco.to_csv('idiscoplus_antibody_notes_already_cleared_asof_2020Nov12.csv')

In [ ]:
imaging_channel_contents & \
    {'username': 'ahoag',
     'request_name': 'test123',
     'sample_name': 'test123-001',
     'imaging_request_number': 1,
     'image_resolution': '1.3x',
     'channel_name': '488'}

In [ ]:
clearing_batch_contents.fetch(as_dict=True,limit=2)

In [ ]:
clearing_batch_contents.fetch?

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples4"' 

In [ ]:
insert_dict = {'username': 'ahoag', 'request_name': 'test_many_samples4',
               'imaging_request_number': 1, 'image_resolution': '1.1x',
               'channel_name': '555',
               'injection_detection': 1,
               'sample_name': 'test_many_samples4-001'}


In [ ]:
imaging_merged_contents = (imaging_request_contents \
    * clearing_batch_contents * sample_contents * request_contents * imaging_channel_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species',
    'image_resolution','channel_name',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)') & \
    {'username':'ahoag',}

In [ ]:
replicated_args = dict(username='username',species='species',
                       datetime_submitted='datetime_submitted',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
    imaging_merged_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',**replicated_args,
    image_resolution='image_resolution').\
    proj(comb_col='CONCAT(image_resolution,",",new_col)',**replicated_args,)
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    **replicated_args,
    final_col='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
dj.U('request_name','imaging_request_number','final_col').aggr(interm2,                               
    **replicated_args,
    number_in_batch='COUNT(*)')

In [ ]:
replicated_args = dict(username='username',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
                        imaging_channel_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',
                        image_resolution='image_resolution').\
                        proj(comb_col='CONCAT(image_resolution,",",new_col)')
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    sample_name='MIN(sample_name)',imaging_str='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
interm3=dj.U('request_name','imaging_request_number','imaging_str').aggr(interm2,                               
    number_in_batch='COUNT(*)',).proj('request_name','number_in_batch')
interm3

In [ ]:
request_counter_dict = {}

In [ ]:
interm3.fetch(as_dict=True)

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples3"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents = db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents.fetch?

In [ ]:
db_lightsheet.Request.ImagingChannel().insert(channel_entry_dicts,skip_duplicates=True)

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
sorted(set(["a","b","a","a"]))

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ProcessingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents 

In [ ]:
var=4
s = f'{{var}} is the number'
s

In [ ]:
b = '1.3x_ventral_up'

In [ ]:
b if 'ventral_up' not in b else b.strip("_ventral_up")

In [ ]:
b.strip("_ventral_up")

In [ ]:
l=[{'id':'A','image_resolution':'4x','ventral_up':True},
   {'id':'B','image_resolution':'4x','ventral_up':False},
    {'id':'C','image_resolution':'1.3x','ventral_up':False},
    {'id':'D','image_resolution':'1.3x','ventral_up':False},
    {'id':'E','image_resolution':'1.3x','ventral_up':True}
  ]
tup_list = [(d['image_resolution'],d['ventral_up']) for d in l]
sorted(set(tup_list))

In [ ]:
all_channel_contents = imaging_channel_contents & 'request_name="test_dorsal_ventral"'

In [ ]:
results_list=all_channel_contents.fetch('image_resolution','ventral_up',as_dict=True)
tup_list = [(d['image_resolution'],d['ventral_up']) for d in results_list]
set(tup_list)
# all_channel_contents.fetch

In [ ]:
for a,b, in tup_list:
    print(a,b)

In [ ]:
bool(int('0'))

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
job_contents

In [ ]:
unique_contents = dj.U('jobid_step0','username',).aggr(
		job_contents,timestamp='max(timestamp)')*job_contents
unique_contents

In [ ]:
int(False)

In [ ]:
int(bool("False"))

In [ ]:
bool("False")

In [ ]:
s='Invitrogen rabbit polyclonal aldolase C'
len(s)

In [ ]:
b='Invitrogen donkey anti rabbit AlexaFluor 647'
len(b)

In [ ]:
c='1:50, 1:100, 1:200'
len(c)

In [ ]:
d={
    'date': datetime.date(2021, 1, 25),
    'brief_descriptor': 'aldolase C titration',
    'animal_model': 'mouse',
    'primary_antibody': 'Invitrogen rabbit polyclonal aldolase C',
    'primary_concentration': '1:50, 1:100, 1:200',
    'primary_order_info': 'Invitrogen PA512317, lot V13073112', 'secondary_antibody': 'Invitrogen donkey anti rabbit AlexaFluor 647', 'secondary_concentration': '1:200', 'secondary_order_info': 'Invitrogen A-31573', 'notes': 'waiting for titration decision'}

In [ ]:
date = '2020-02-01'
datetime.strptime(date,'%Y-%m-%d')

In [ ]:
[1,2]+[3]

In [ ]:
sample_imaging_dict = {}

## Debugging request_overview 

In [ ]:
username='ahoag'
request_name='test_imaging_batches_clearing_batches'
request_contents = db_lightsheet.Request() & f'request_name="{request_name}"' & \
            f'username="{username}"'
request_contents = request_contents.proj('description','species','number_of_samples',
    'is_archival',datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
sample_contents = db_lightsheet.Request.Sample() & f'request_name="{request_name}"' & f'username="{username}"' 
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & \
f'request_name="{request_name}"' & f'username="{username}"'
clearing_batch_sample_contents = db_lightsheet.Request.ClearingBatchSample() & \
f'request_name="{request_name}"' & f'username="{username}"' 
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 
processing_request_contents = db_lightsheet.Request.ProcessingRequest() & \
 f'request_name="{request_name}"' & f'username="{username}"' 

In [ ]:
replicated_args = dict(number_of_samples='number_of_samples',description='description',
        species='species',clearing_progress='clearing_progress',
        clearer='clearer',
        imaging_progress='imaging_progress',imager='imager',
        is_archival='is_archival',
        link_to_clearing_spreadsheet='link_to_clearing_spreadsheet')

In [ ]:
sample_joined_contents = request_contents * clearing_batch_sample_contents * clearing_batch_contents.proj(
    'clearing_progress','clearer','link_to_clearing_spreadsheet')
imaging_joined_contents = sample_joined_contents.aggr(
        imaging_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        n_imaged='CONVERT(SUM(imaging_progress="complete"),char)',
        total_imaging_requests='COUNT(*)',
        keep_all_rows=True
        ).proj(**replicated_args,
               total_imaging_requests='IF(n_imaged is NULL, "0",total_imaging_requests)',
               imaging_request_number='IF(imaging_request_number is NULL, "N/A",imaging_request_number)'
            # fraction_imaged='CONCAT(n_imaged,"/",total_imaging_requests)'
            )
processing_joined_contents = (dj.U('username','request_name') * imaging_joined_contents).aggr(   
        processing_request_contents,
        **replicated_args,
        imaging_request_number='imaging_request_number',
        processing_request_number='processing_request_number',
        processor='processor',
        processing_progress='processing_progress',
        total_imaging_requests='total_imaging_requests',
        n_processed='CONVERT(SUM(processing_progress="complete"),char)',
        total_processing_requests='CONVERT(COUNT(processing_progress),char)',
        keep_all_rows=True
        ).proj(
            **replicated_args,
            processing_request_number='IF(processing_request_number is NULL, "N/A",processing_request_number)',
            processor='processor',
            processing_progress='processing_progress',
            total_imaging_requests='total_imaging_requests',
            total_processing_requests='IF(n_processed is NULL,0,total_processing_requests)', 
            )

In [ ]:
sample_joined_contents

In [ ]:
imaging_joined_contents

In [ ]:
processing_joined_contents

## Debugging imaging_manager with microscope column

In [26]:
request_contents = db_lightsheet.Request() & {'username':'hboele'}
sample_contents = db_lightsheet.Request.Sample() & {'username':'hboele'}
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() & {'username':'hboele'}
imaging_batch_contents = db_lightsheet.Request.ImagingBatch() & {'username':'hboele'}
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() & {'username':'hboele'}


imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents*imaging_resolution_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    'microscope',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)')
imaging_request_contents = dj.U('username','request_name',
    'clearing_batch_number',
    'imaging_batch_number','imaging_request_number').aggr(
    imaging_request_contents,clearer='clearer',
    clearing_progress='clearing_progress',
    datetime_submitted='datetime_submitted',
    imaging_progress='imaging_progress',imager='imager',
    species='species',number_in_imaging_batch='number_in_imaging_batch',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)',
    all_samples_lavision='SUM(IF(microscope="lavision",1,0))=count(*)',
    all_samples_smartspim='SUM(IF(microscope="smartspim",1,0))=count(*)')   
imaging_request_contents

username user in the lab,request_name,clearing_batch_number,imaging_batch_number,imaging_request_number,clearing_progress,clearer user in the lab,species,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_progress,datetime_submitted calculated attribute,all_samples_cleared calculated attribute,all_samples_lavision calculated attribute,all_samples_smartspim calculated attribute
hboele,HJB_DCN_YFP,1,1,1,complete,ll3,mouse,aichen,2,complete,2020-11-16 21:34:36,1,0,1
hboele,HJB_DCN_YFP,2,1,1,in progress,ll3,mouse,aichen,6,incomplete,2020-11-16 21:34:36,0,0,1


In [25]:
imaging_resolution_contents & {'username':'hboele'}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,microscope,notes_for_imager,notes_from_imaging
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,3.6x,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,3.6x,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16685-01M,1,3.6x,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16686-01F,1,3.6x,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16686-02F,1,3.6x,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16687-01F,1,3.6x,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16687-02F,1,3.6x,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,


## Debugging pyalign manager
In order for a channel to be ready to be "pystriped" it needs to have been stitched. Therefore it needs to have an entry in the SmartspimStitchedChannel() table. 


In [34]:
imaging_request_contents = db_lightsheet.Request.ImagingRequest() 
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() 
imaging_channel_contents = db_lightsheet.Request.ImagingChannel()
smartspim_combined_contents = imaging_request_contents * imaging_channel_contents *\
    imaging_resolution_contents & {'imaging_progress':'complete','microscope':'SmartSPIM'}
smartspim_combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress,microscope,notes_for_imager,notes_from_imaging
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,488,0,None,2020-10-27,10:10:45,2020-12-21,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,647,0,None,2020-10-27,10:10:45,2020-12-21,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,488,0,None,2020-10-27,10:10:45,2020-12-22,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,647,0,None,2020-10-27,10:10:45,2020-12-22,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,488,0,aichen,2020-12-03,13:56:31,2021-01-06,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,images were blurry due to movement of the sample during movement. Reimaging was requested
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,647,0,aichen,2020-12-03,13:56:31,2021-01-06,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,images were blurry due to movement of the sample during movement. Reimaging was requested
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,488,0,aichen,2021-01-16,15:43:13,2021-01-19,complete,None,nan,1,1,1,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3900,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,647,0,aichen,2021-01-16,15:43:13,2021-01-19,complete,None,nan,1,1,0,1,0,0,1,None,horizontal,nan,3x5,0.0,2.0,3900,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,488,0,None,2020-10-22,12:56:41,2020-12-01,complete,None,1.0,1,1,1,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,647,0,None,2020-10-22,12:56:41,2020-12-01,complete,None,1.0,1,1,0,0,0,1,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,


In [35]:
# First need to create ProcessingRequest() entries for these if they don't exist
for entry in smartspim_combined_contents:
    processing_request_insert_dict = {}
    processing_request_insert_dict['username'] = entry['username']
    processing_request_insert_dict['request_name'] = entry['request_name']
    processing_request_insert_dict['sample_name'] = entry['sample_name']
    processing_request_insert_dict['imaging_request_number'] = entry['imaging_request_number']
    processing_request_insert_dict['processing_request_number'] = 1
    processing_request_insert_dict['processor'] = 'ahoag'
    processing_request_insert_dict['processing_request_date_submitted'] = entry['imaging_request_date_submitted']
    processing_request_insert_dict['processing_request_time_submitted'] = entry['imaging_request_time_submitted']
    processing_request_insert_dict['processing_progress'] = 'incomplete'
    db_lightsheet.Request.ProcessingRequest().insert1(processing_request_insert_dict,skip_duplicates=True)

In [36]:
# Now create ProcessingResolutionRequest() entries for these if they don't exist
for entry in smartspim_combined_contents:
    processing_resolution_request_insert_dict = {}
    processing_resolution_request_insert_dict['username'] = entry['username']
    processing_resolution_request_insert_dict['request_name'] = entry['request_name']
    processing_resolution_request_insert_dict['sample_name'] = entry['sample_name']
    processing_resolution_request_insert_dict['imaging_request_number'] = entry['imaging_request_number']
    processing_resolution_request_insert_dict['processing_request_number'] = 1
    processing_resolution_request_insert_dict['image_resolution'] = entry['image_resolution']
    processing_resolution_request_insert_dict['ventral_up'] = entry['ventral_up']
    processing_resolution_request_insert_dict['atlas_name'] = 'princeton_mouse_atlas'
    processing_resolution_request_insert_dict['final_orientation'] = 'sagittal'
    db_lightsheet.Request.ProcessingResolutionRequest().insert1(
        processing_resolution_request_insert_dict,skip_duplicates=True)

In [37]:
# Now create SmartSpimStitchedChannel() entries for these if they don't exist
processing_resolution_request_contents = db_lightsheet.Request.ProcessingResolutionRequest() 
smartspim_combined_contents = imaging_request_contents * imaging_channel_contents *\
    imaging_resolution_contents & {'imaging_progress':'complete','microscope':'SmartSPIM'}
smartspim_combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress,microscope,notes_for_imager,notes_from_imaging
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,488,0,None,2020-10-27,10:10:45,2020-12-21,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,647,0,None,2020-10-27,10:10:45,2020-12-21,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,488,0,None,2020-10-27,10:10:45,2020-12-22,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,647,0,None,2020-10-27,10:10:45,2020-12-22,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,488,0,aichen,2020-12-03,13:56:31,2021-01-06,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,images were blurry due to movement of the sample during movement. Reimaging was requested
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,647,0,aichen,2020-12-03,13:56:31,2021-01-06,complete,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,images were blurry due to movement of the sample during movement. Reimaging was requested
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,488,0,aichen,2021-01-16,15:43:13,2021-01-19,complete,None,nan,1,1,1,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3900,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,647,0,aichen,2021-01-16,15:43:13,2021-01-19,complete,None,nan,1,1,0,1,0,0,1,None,horizontal,nan,3x5,0.0,2.0,3900,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,488,0,None,2020-10-22,12:56:41,2020-12-01,complete,None,1.0,1,1,1,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,647,0,None,2020-10-22,12:56:41,2020-12-01,complete,None,1.0,1,1,0,0,0,1,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,


In [39]:
# Now create ProcessingResolutionRequest() entries for these if they don't exist
for entry in smartspim_combined_contents:
    smartspim_stitched_channel_insert_dict = {}
    smartspim_stitched_channel_insert_dict['username'] = entry['username']
    smartspim_stitched_channel_insert_dict['request_name'] = entry['request_name']
    smartspim_stitched_channel_insert_dict['sample_name'] = entry['sample_name']
    smartspim_stitched_channel_insert_dict['imaging_request_number'] = entry['imaging_request_number']
    smartspim_stitched_channel_insert_dict['processing_request_number'] = 1
    smartspim_stitched_channel_insert_dict['image_resolution'] = entry['image_resolution']
    smartspim_stitched_channel_insert_dict['channel_name'] = entry['channel_name']
    smartspim_stitched_channel_insert_dict['ventral_up'] = entry['ventral_up']
    smartspim_stitched_channel_insert_dict['datetime_stitching_started'] = '2021-01-01 00:00:00'
    smartspim_stitched_channel_insert_dict['datetime_stitching_completed'] = '2021-01-01 1:00:00'
    db_lightsheet.Request.SmartspimStitchedChannel().insert1(
        smartspim_stitched_channel_insert_dict,skip_duplicates=True)

In [80]:
request_contents = db_lightsheet.Request() 
sample_contents = db_lightsheet.Request.Sample() 
clearing_batch_contents = db_lightsheet.Request.ClearingBatch() 
imaging_batch_contents = db_lightsheet.Request.ImagingBatch() 
imaging_batch_sample_contents = db_lightsheet.Request.ImagingBatchSample() 

imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() # to get microscope
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() 

smartspim_imaging_batch_contents = (imaging_batch_sample_contents *\
                          imaging_channel_contents * imaging_resolution_contents) & 'microscope="smartspim"'
smartspim_imaging_batch_contents

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_batch_number,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress,microscope,notes_for_imager,notes_from_imaging
cz15,zimmerman_01,zimmerman_01-001,1,1,1,3.6x,488,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-001,1,1,1,3.6x,647,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,1,1,3.6x,488,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_01,zimmerman_01-002,1,1,1,3.6x,647,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,10.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,I wasn't sure how to complete this table - please reach out to me before imaging.,
cz15,zimmerman_02,zimmerman_02-f11,1,1,1,3.6x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,
cz15,zimmerman_02,zimmerman_02-f11,1,1,1,3.6x,642,0,None,nan,1,1,0,0,0,1,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,
cz15,zimmerman_02,zimmerman_02-f12,1,1,1,3.6x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,
cz15,zimmerman_02,zimmerman_02-f12,1,1,1,3.6x,642,0,None,nan,1,1,0,0,0,1,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,
cz15,zimmerman_02,zimmerman_02-f13,1,1,1,3.6x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,
cz15,zimmerman_02,zimmerman_02-f13,1,1,1,3.6x,642,0,None,nan,1,1,0,0,0,1,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Please use the same imaging parameters/protocol as for the Nov 2020 pilot brains.,


In [83]:
dj.U('username','request_name',
    'imaging_batch_number','imaging_request_number').aggr(
    smartspim_imaging_batch_contents,
    n_channels = 'count(channel_name)',
    )

username user in the lab,request_name,imaging_batch_number,imaging_request_number,n_channels calculated attribute
cz15,zimmerman_01,1,1,4
cz15,zimmerman_02,1,1,24
diamanti,SynGCaMP7_12,1,1,2
diamanti,SynGCaMP7_12,1,2,2
gerardjb,Aldolase_C_Titration,1,1,6
hboele,HJB_DCN_YFP,1,1,9
pbibawi,pb_udisco_647,1,1,9
pbibawi,pb_udisco_647_488_4x,1,1,18


In [118]:
stitching_contents = db_lightsheet.Request.SmartspimStitchedChannel() & {'username':'ahoag'}
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() & {'username':'ahoag'}
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & {'username':'ahoag'}
imaged_channel_contents = imaging_channel_contents * imaging_request_contents & {'imaging_progress':'complete'}
pystripe_contents = db_lightsheet.Request.SmartspimPystripeChannel() & {'username':'ahoag'}

In [119]:
stitching_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,datetime_stitching_started,datetime_stitching_completed,smartspim_stitching_spock_jobid the jobid from the final stitching step for SmartSPIM,smartspim_stitching_spock_job_progress the spock job status code for the final step in the light sheet processing pipeline,brainpipe_commit the commit that is checked out on the machine at the time the job was submitted
ahoag,test_3p6x,test_3p6x-001,1,3.6x,488,0,1,2021-01-01 00:00:00,None,None,None,None
ahoag,test_3p6x,test_3p6x-001,1,3.6x,555,0,1,2021-01-01 00:00:00,None,None,None,None


In [120]:
imaged_channel_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress
ahoag,test_3p6x,test_3p6x-001,1,3.6x,488,0,None,nan,1,1,1,0,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,ahoag,2021-02-22,15:16:27,None,complete
ahoag,test_3p6x,test_3p6x-001,1,3.6x,555,0,None,nan,1,1,0,1,0,0,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,ahoag,2021-02-22,15:16:27,None,complete
ahoag,test_3p6x,test_3p6x-001,1,3.6x,642,0,None,nan,1,1,0,0,0,1,0,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,ahoag,2021-02-22,15:16:27,None,complete
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-001,1,1.1x,488,0,None,1.0,1,0,1,0,0,0,0,None,horizontal,nan,1x1,0.0,10.0,657,test488,0,20994424,SUBMITTED,None,None,ahoag,2021-02-05,12:02:59,2021-02-05,complete
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-002,1,1.1x,488,0,None,1.0,1,0,1,0,0,0,0,None,horizontal,nan,1x1,0.0,10.0,657,test488,0,20994432,SUBMITTED,None,None,ahoag,2021-02-05,12:02:59,2021-02-05,complete


In [123]:
stitched_aggr_contents = dj.U('username','request_name','sample_name','imaging_request_number').aggr(stitching_contents,
    n_channels_stitched='count(*)')
stitched_aggr_contents

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_stitched calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,2


In [128]:
imaged_aggr_contents = dj.U('username','request_name','sample_name','imaging_request_number').aggr(
    imaged_channel_contents,
    n_channels_imaged='count(*)')
imaged_aggr_contents

/home/ahoag/anaconda3/envs/dj/lib/python3.8/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,3
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-001,1,1
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-002,1,1


In [127]:
imaged_aggr_contents * stitched_aggr_contents & 'n_channels_imaged=n_channels_stitched'

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute


In [103]:
stitching_contents * imaging_request_contents & {'imaging_progress':'complete'}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,datetime_stitching_started,datetime_stitching_completed,smartspim_stitching_spock_jobid the jobid from the final stitching step for SmartSPIM,smartspim_stitching_spock_job_progress the spock job status code for the final step in the light sheet processing pipeline,brainpipe_commit the commit that is checked out on the machine at the time the job was submitted,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,3.6x,488,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None,None,2020-11-16,21:34:36,2020-12-22,complete
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,3.6x,647,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None,None,2020-11-16,21:34:36,2020-12-22,complete
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,3.6x,488,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None,None,2020-11-16,21:34:36,2020-12-22,complete
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,3.6x,647,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None,None,2020-11-16,21:34:36,2020-12-22,complete


In [99]:
dj.U('username','request_name','sample_name','imaging_request_number').aggr(stitching_contents,
    n_channels='count(*)')

username user in the lab,request_name,sample_name,imaging_request_number,n_channels calculated attribute
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,2
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,2


In [90]:
stitching_contents - pystripe_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,datetime_stitching_started,datetime_stitching_completed,smartspim_stitching_spock_jobid the jobid from the final stitching step for SmartSPIM,smartspim_stitching_spock_job_progress the spock job status code for the final step in the light sheet processing pipeline,brainpipe_commit the commit that is checked out on the machine at the time the job was submitted
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,3.6x,488,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,3.6x,647,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,3.6x,488,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,3.6x,647,0,1,2021-01-01 00:00:00,2021-01-01 01:00:00,None,None,None


In [81]:
smartspim_imaging_batch_contents & {'username':'hboele'}

username user in the lab,request_name,sample_name,clearing_batch_number,imaging_batch_number,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress,microscope,notes_for_imager,notes_from_imaging
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,1,1,3.6x,488,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,1,1,3.6x,647,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3600,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,1,1,3.6x,488,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3300,Ex_488_Em_0,0,None,None,None,None,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,1,1,3.6x,647,0,None,1.0,1,1,0,0,0,0,0,None,horizontal,nan,3x5,0.0,2.0,3300,Ex_642_Em_2,0,None,None,None,None,SmartSPIM,,
hboele,HJB_DCN_YFP,20-MI-16685-01M,2,1,1,3.6x,488,0,None,nan,1,1,1,1,1,1,1,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16686-01F,2,1,1,3.6x,488,0,None,nan,1,1,1,1,1,1,1,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16686-02F,2,1,1,3.6x,488,0,None,nan,1,1,1,1,1,1,1,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16687-01F,2,1,1,3.6x,488,0,None,nan,1,1,1,1,1,1,1,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,
hboele,HJB_DCN_YFP,20-MI-16687-02F,2,1,1,3.6x,488,0,None,nan,1,1,1,1,1,1,1,None,sagittal,nan,1x1,0.0,10.0,None,None,None,None,None,None,None,SmartSPIM,Henk-Jan wants 642 channel imaged as well as 488,


In [271]:
imaging_request_contents = db_lightsheet.Request.ImagingRequest() 
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() 
imaged_channel_contents = imaging_channel_contents * imaging_request_contents &\
    {'imaging_progress':'complete'} & 'image_resolution="3.6x"'

stitching_contents = db_lightsheet.Request.SmartspimStitchedChannel()

imaged_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(
    imaged_channel_contents,
    n_channels_imaged='count(*)')

stitched_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number',
    'processing_request_number').aggr(
    stitching_contents,
    n_channels_stitched='count(*)')

imaged_and_stitched_contents = imaged_aggr_contents * stitched_aggr_contents &\
    'n_channels_imaged=n_channels_stitched'

pystripe_channel_contents = db_lightsheet.Request.SmartspimPystripeChannel() 

# stitched_and_pystriped_contents = imaged_and_stitched_contents * pystripe_channel_contents
combined_contents = imaged_and_stitched_contents * pystripe_channel_contents

pystripe_imaging_requests = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(combined_contents,
                                   username='username',
                                   n_channels_imaged='n_channels_imaged',
                                   n_channels_pystriped='sum(pystripe_performed)',
                                   n_channels_started='count(*)',
                                  )
pystripe_imaging_requests



username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_pystriped calculated attribute,n_channels_started calculated attribute


In [272]:
imaging_aggr_contents

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,6
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-001,1,1
ahoag,test_imaging_batches_clearing_batches,test_imaging_batches_clearing_batches-002,1,1


In [273]:
stitched_aggr_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,n_channels_stitched calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,1,6


In [274]:
imaged_and_stitched_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,1,6,6


In [275]:
pystripe_channel_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe


In [265]:
pystripe_imaging_requests &\
    'n_channels_imaged!=n_channels_pystriped' & 'n_channels_started>0' 

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_pystriped calculated attribute,n_channels_started calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,3,2,3


In [266]:
pystripe_imaging_requests &\
    'n_channels_imaged=n_channels_pystriped'  

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_pystriped calculated attribute,n_channels_started calculated attribute


In [248]:
imaged_and_stitched_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,1,3,3


In [249]:
pystripe_channel_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe
ahoag,test_3p6x,test_3p6x-001,1,3.6x,488,0,1,flat.tiff,1,2021-03-24 10:04:33,None,21320219,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,3.6x,561,0,1,flat.tiff,1,2021-03-24 10:05:21,None,21320221,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,3.6x,642,0,1,flat.tiff,0,2021-03-24 10:20:47,None,21320227,COMPLETED


In [222]:
combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,488,0,3,3,flat.tiff,1,2021-03-24 10:04:33,None,21320219,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,561,0,3,3,flat.tiff,1,2021-03-24 10:05:21,None,21320221,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,642,0,3,3,flat.tiff,0,2021-03-24 10:20:47,None,21320227,COMPLETED


In [221]:
pystripe_imaging_requests & 'any_channels_started > 0'

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_pystriped calculated attribute,n_channels_started calculated attribute,all_channels_pystriped calculated attribute,any_channels_started calculated attribute


In [237]:
schema = dj.Schema('ahoag_test')
@schema
class Task(dj.Manual):
    definition = """
    sample_name: varchar(32)
    channel_name: varchar(32)
    ---
    pystripe_performed: boolean 
    """
@schema
class Channel(dj.Manual):
    definition = """
    sample_name: varchar(32)
    channel_name: varchar(32) 
    ---
    """

In [250]:
channel_insert_list = [
    {'sample_name':'sample1',
     'channel_name':'channel1'},
    {'sample_name':'sample1',
     'channel_name':'channel2'},
    {'sample_name':'sample1',
     'channel_name':'channel3'}
]
task_insert_list = [
    {'sample_name':'sample1',
     'channel_name':'channel1',
     'pystripe_performed':0},
    {'sample_name':'sample1',
     'channel_name':'channel2',
     'pystripe_performed':1}
]
Channel().insert(channel_insert_list,skip_duplicates=True)
Task().insert(task_insert_list,skip_duplicates=True)

In [251]:
Channel()

sample_name,channel_name
sample1,channel1
sample1,channel2
sample1,channel3


In [252]:
Task()

sample_name,channel_name,pystripe_performed
sample1,channel1,0
sample1,channel2,1


In [256]:
imaged_aggr_contents = dj.U('sample_name').aggr(Channel(),n_channels_imaged='count(*)')
imaged_aggr_contents

sample_name,n_channels_imaged calculated attribute
sample1,3


In [257]:
combined_contents = imaged_aggr_contents * Task()
combined_contents

sample_name,channel_name,n_channels_imaged calculated attribute,pystripe_performed
sample1,channel1,3,0
sample1,channel2,3,1


In [260]:
result = dj.U('sample_name').aggr(combined_contents,
                                   sample_name='sample_name',
                                   n_channels_imaged='n_channels_imaged',
                                   n_channels_pystriped='sum(pystripe_performed)',
                                   n_channels_started='count(*)',
                                  ).proj('sample_name',
                                         'n_channels_pystriped',
                                         'n_channels_started',
                                         all_channels_pystriped='n_channels_imaged=n_channels_pystriped',
                                         any_channels_started='n_channels_started>0')
result

sample_name,n_channels_pystriped calculated attribute,n_channels_started calculated attribute,all_channels_pystriped calculated attribute,any_channels_started calculated attribute
sample1,1,2,0,1


In [262]:
result & 'any_channels_started>0'


sample_name,n_channels_pystriped calculated attribute,n_channels_started calculated attribute,all_channels_pystriped calculated attribute,any_channels_started calculated attribute


In [189]:
combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,488,0,3,3,flat.tiff,1,2021-03-24 10:04:33,None,21320219,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,561,0,3,3,flat.tiff,1,2021-03-24 10:05:21,None,21320221,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,642,0,3,3,flat.tiff,1,2021-03-24 10:20:47,None,21320227,COMPLETED


username user in the lab,request_name,sample_name,imaging_request_number,all_channels_pystriped calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,1


In [170]:
combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,488,0,3,3,flat.tiff,1,2021-03-24 10:04:33,None,21320219,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,1,3.6x,561,0,3,3,flat.tiff,1,2021-03-24 10:05:21,None,21320221,COMPLETED


In [168]:
imaged_and_stitched_contents

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute
ahoag,test_3p6x,test_3p6x-001,1,1,3,3


In [167]:
pystripe_channel_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe
ahoag,test_3p6x,test_3p6x-001,1,3.6x,488,0,1,flat.tiff,1,2021-03-24 10:04:33,None,21320219,COMPLETED
ahoag,test_3p6x,test_3p6x-001,1,3.6x,561,0,1,flat.tiff,1,2021-03-24 10:05:21,None,21320221,COMPLETED


In [160]:
imaging_requests_currently_being_pystriped

username user in the lab,request_name,sample_name,imaging_request_number,processing_request_number
ahoag,test_3p6x,test_3p6x-001,1,1


In [49]:
imaging_request_contents = db_lightsheet.Request.ImagingRequest() 
imaging_channel_contents = db_lightsheet.Request.ImagingChannel() 
imaged_channel_contents = imaging_channel_contents * imaging_request_contents &\
    {'imaging_progress':'complete'} & 'image_resolution="3.6x"'

stitching_contents = db_lightsheet.Request.SmartspimStitchedChannel() 

imaged_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(
    imaged_channel_contents,
    n_channels_imaged='count(*)')

stitched_aggr_contents = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(
    stitching_contents,
    n_channels_stitched='sum(smartspim_stitching_spock_job_progress="COMPLETED")')

imaged_and_stitched_contents = imaged_aggr_contents * stitched_aggr_contents &\
    'n_channels_imaged=n_channels_stitched'

pystripe_channel_contents = db_lightsheet.Request.SmartspimPystripeChannel() 

''' Get contents ready to be pystriped.
These are entries of the stitching table that are not in the pystripe table yet. '''

contents_ready_to_pystripe = imaged_and_stitched_contents - pystripe_channel_contents

In [50]:
imaged_and_stitched_contents

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute
cz15,zimmerman_01,zimmerman_01-001,1,2,2
cz15,zimmerman_01,zimmerman_01-002,1,2,2
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,2,2
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,2,2
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,2,2
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_200,1,2,2
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_50,1,2,2
hboele,HJB_DCN_YFP,20-MI-16684-01M,1,2,2
hboele,HJB_DCN_YFP,20-MI-16684-02M,1,2,2
pbibawi,pb_udisco,pb_udisco_M122,1,2,2


In [51]:
combined_contents = imaged_and_stitched_contents * pystripe_channel_contents
''' Figure out which imaging requests are currently being pystriped, ready to be pystriped
and already completed being pystriped '''
pystripe_imaging_requests = dj.U('username','request_name','sample_name',
    'imaging_request_number').aggr(combined_contents,
                                   username='username',
                                   n_channels_imaged='n_channels_imaged',
                                   n_channels_pystriped='sum(pystripe_performed)',
                                   n_channels_started='count(*)',
                                  )

In [52]:
combined_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,n_channels_imaged calculated attribute,n_channels_stitched calculated attribute,flatfield_filename the name of the flat field file generated by the FlatGenerate GUI on smartspim computer,pystripe_performed whether pystripe has been run on this channel,datetime_pystripe_started,datetime_pystripe_completed,smartspim_pystripe_spock_jobid the jobid from running pystripe,smartspim_pystripe_spock_job_progress the spock job status code for pystripe,smartspim_corrected_precomputed_spock_jobid,smartspim_corrected_precomputed_spock_job_progress
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
cz15,zimmerman_01,zimmerman_01-001,1,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
cz15,zimmerman_01,zimmerman_01-002,1,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,1,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,488,0,2,2,flat.tiff,1,None,None,None,None,None,None
gerardjb,Aldolase_C_Titration,Aldolase_C_Titration-1_100,1,3.6x,642,0,2,2,flat.tiff,1,None,None,None,None,None,None


In [39]:
pystripe_imaging_requests

username user in the lab,request_name,sample_name,imaging_request_number,n_channels_imaged calculated attribute,n_channels_pystriped calculated attribute,n_channels_started calculated attribute


In [53]:
bstr = b'\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3J\x1b[H\x1b[2J\x1b[3Jin VolumeConverter::generateTiles(path = "/jukebox/LightSheetData/lightserv/pbibawi/pb_udisco/pb_udisco_M122/imaging_request_1/rawdata/resolution_3.6x/Ex_488_Em_0_stitched/", resolutions = 1000000000, slice_height = -1, slice_width = -1, method = 1, show_progress_bar = true, saved_img_format = tif, saved_img_depth = 16, frame_dir = "")\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tInitializing...\nTIME REMAINING:\t0 minutes and 0 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating slices from 0 to 1 og 2302\nTIME REMAINING:\t153 minutes and 23 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating resolution 1 of 1\nTIME REMAINING:\t153 minutes and 23 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tSaving to disc resolution 1\nTIME REMAINING:\t153 minutes and 23 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating slices from 1 to 2 og 2302\nTIME REMAINING:\t172 minutes and 29 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating resolution 1 of 1\nTIME REMAINING:\t172 minutes and 29 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tSaving to disc resolution 1\nTIME REMAINING:\t172 minutes and 29 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating slices from 2 to 3 og 2302\nTIME REMAINING:\t191 minutes and 35 seconds\nPROGRESS:\t0%\t::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n\nOPERATION:\tMultiresolution tile generation\nPHASE:\t\tGenerating resolution 1 of 1\nTIME REMAINING:\t191 minutes and 35 seconds\nPROGRESS:\t0%\t'

In [59]:
bstr.decode('string_escape')

LookupError: unknown encoding: string_escape

In [3]:
import os
os.system('df -h')

0

In [27]:
import subprocess
result = subprocess.run('df -BG | grep LightSheetData',shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8')
avail_str = result.split()[3]
avail_GB = int(avail_str[:-1])
avail_GB

1394

In [25]:
result.split()

['//bucket.pni.princeton.edu/LightSheetData',
 '122880G',
 '121487G',
 '1394G',
 '99%',
 '/home/ahoag/mounts/LightSheetData']

In [32]:
db_lightsheet.Request.ImagingChannel() & {'image_resolution':"3.6x","injection_detection":1}

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress
diamanti,SynGCaMP7_12,SynGCaMP7_12-001,2,3.6x,642,0,None,nan,1,1,0,1,0,0,1,None,horizontal,nan,3x5,0.0,2.0,3900,Ex_642_Em_2,0,None,None,None,None
